## Purpose

Filter and calculate means of monthly mixed layer depth, temperature, X and Y seawater velocities from MOM6 for yellowtail.

## Set up

In [1]:
import xarray as xr
import numpy as np
import pandas as pd
import time

## Download data

Script assumes you have these files in a `OcModels/Data/MOM6` folder. To download necessary inputs, run:

*Depth (2-d)*
* *deptho* `wget -O ocean_static.deptho.nc https://psl.noaa.gov/thredds/fileServer/Projects/CEFI/regional_mom6/cefi_derivative/northeast_pacific/full_domain/hindcast/monthly/regrid/r20250509/static/ocean_static.deptho.nc`

*Mixed layer depth (2-d)*
* *MLD_003* `wget -O MLD_003.nep.full.hcast.monthly.regrid.r20250509.199301-202412.nc https://psl.noaa.gov/thredds/fileServer/Projects/CEFI/regional_mom6/cefi_portal/northeast_pacific/full_domain/hindcast/monthly/regrid/r20250509/MLD_003.nep.full.hcast.monthly.regrid.r20250509.199301-202412.nc`

*Temperature (3-d)*
* *thetao* `wget -O thetao.nep.full.hcast.monthly.regrid.r20250509.199301-202412.nc https://psl.noaa.gov/thredds/fileServer/Projects/CEFI/regional_mom6/cefi_portal/northeast_pacific/full_domain/hindcast/monthly/regrid/r20250509/thetao.nep.full.hcast.monthly.regrid.r20250509.199301-202412.nc`

*X & Y sea water velocities (3-d)*
* *uo_rotate* `wget -O uo_rotate.nep.full.hcast.monthly.regrid.r20250509.199301-202412.nc https://psl.noaa.gov/thredds/fileServer/Projects/CEFI/regional_mom6/cefi_portal/northeast_pacific/full_domain/hindcast/monthly/regrid/r20250509/uo_rotate.nep.full.hcast.monthly.regrid.r20250509.199301-202412.nc`
* *vo_rotate* `wget -O vo_rotate.nep.full.hcast.monthly.regrid.r20250509.199301-202412.nc https://psl.noaa.gov/thredds/fileServer/Projects/CEFI/regional_mom6/cefi_portal/northeast_pacific/full_domain/hindcast/monthly/regrid/r20250509/vo_rotate.nep.full.hcast.monthly.regrid.r20250509.199301-202412.nc`

Depth was based on a link Chia-Wei shared with me. The remaining queries were generated on the [CEFI Data Portal](https://psl.noaa.gov/cefi_portal/#data_access).
* Region: NEP
* Subregion: full domain
* Experiment Type: hindcast
* Output Frequency: monthly regrid
* Release: r20250509
* Data Category: ocean_monthly_z (for 3-d) or ocean_monthly (for 2-d)

In the future, it'd be nice to do this on the raw data rather than the gridded data.

## Life stage dictionary

Based on [Data Availability](https://docs.google.com/document/d/1P8D0kH2xn4NYBc0ib3rYfSO0KAiyDNQ_kZgG4Qub7qY/edit?tab=t.0#heading=h.z5x77oqxpj4i), define filters for each lifestage for time, bottom depth, and latitude.

For northern yellowtail, the bottom depth is used to determine the longitudinal extent, and the average is taken across the whole water column. For example, if the longitudinal extent is bottom depths 90-180m, the netCDF is filtered for isobaths where the depth is \[90, 180], and the average is calculated for all the water \[0, 180] at those depths.

In [2]:
# assign months, latitudes, and depths to filter
# all are integers, depths are in meters
# filters are inclusive (closed interval), e.g. depth 0-180 includes depths 0 and 180 and every depth between

lifestage_dict = {
    "cop": { # copulation
        "min_month": 8,
        "max_month": 10,
        "min_lat": 40,
        "max_lat": 48,
        "min_depth": 90,
        "max_depth": 180
    }, 
    "part": { # partuition
        "min_month": 1,
        "max_month": 4,
        "min_lat": 40,
        "max_lat": 48,
        "min_depth": 0,
        "max_depth": 180
    },
    "larv": { # larvae
        "min_month": 2,
        "max_month": 3,
        "min_lat": 40,
        "max_lat": 48,
        "min_depth": 0,
        "max_depth": 90
    },
    "pjuv": { # pelagic juvenile
        "min_month": 4,
        "max_month": 8,
        "min_lat": 40,
        "max_lat": 48,
        "min_depth": 30,
        "max_depth": 130
    }
}

## Load datasets

In [3]:
# depth is used to filter longitudinal extent
ds_depth = xr.open_dataset("../../Data/MOM6/ocean_static.deptho.nc")

# mixed layer depth
ds_mld = xr.open_dataset("../../Data/MOM6/MLD_003.nep.full.hcast.monthly.regrid.r20250509.199301-202412.nc")

# temperature
ds_temp = xr.open_dataset("../../Data/MOM6/thetao.nep.full.hcast.monthly.regrid.r20250509.199301-202412.nc")

# X and Y sea water velocities
ds_xv = xr.open_dataset("../../Data/MOM6/uo_rotate.nep.full.hcast.monthly.regrid.r20250509.199301-202412.nc")
ds_yv = xr.open_dataset("../../Data/MOM6/vo_rotate.nep.full.hcast.monthly.regrid.r20250509.199301-202412.nc")

## Functions

In [4]:
'''
Print the size and dimensions of a dataset for a variable named 'var_name'
If nan_bool is true, also count the number of non-NA records (# and %) for that variable

Inputs:
- dataset: xarray Dataset with the variable var_name
- var_name: string with the variable name to count
- nan_bool: boolean, to print record count and % for the variable

Outputs:
- None, will print output
'''

def print_ds_info(dataset, var_name, nan_bool = False):
    denominator = dataset[var_name].size
    print(f'{denominator} records with dimensions {dataset[var_name].shape}')
    
    if nan_bool:
        numerator = np.isfinite(dataset[var_name].data).sum()
        percent = numerator / denominator * 100
        print(f'{numerator} records ({percent:.{2}f}%) are not np.nan')
        
    return None

In the future, it'd be nice to adjust these functions to have an argument to only run if the output file isn't in the directory already, so the notebook could run top to bototm and only re-caculate missing indices.

In [5]:
'''
Calculate annual mean for a 2-D variable

Inputs:
- var_dataset: xarray Dataset with the variable to average
- depth_dataset: xarray Dataset with depth to filter
- ds_var_name: string with the variable name, used to extract from var_dataset
- df_var_name: string with the variable name, used to rename column in returned dataframe
- filter_dict: dictionary with relevant filters
    - min_mon, max_mon: minimum and maximum month (integers, inclusive)
    - min_lat, max_lat: minimum and maximum latitude (integers or floats, inclusive)
    - min_depth, max_depth: minimum and maximum depth to filter the longitudinal extent
- print_counts: boolean, to print record count and % at each step; defaults to True for 2-D

Outputs:
- ds_df: pandas dataframe with an annual average of the variable
'''

def annual_mean_2d(var_dataset, depth_dataset, ds_var_name, df_var_name, filter_dict, print_counts = True):

    # start timer
    start_time = time.time()
    
    print("---" + df_var_name + "---")
    print_ds_info(var_dataset, ds_var_name, print_counts)
    
    # merge files
    ds_merged = xr.merge([var_dataset, depth_dataset])
    print_ds_info(ds_merged, ds_var_name, print_counts)

    # create list with months
    month_list = [x for x in range(filter_dict["min_month"], filter_dict["max_month"]+1)]
    # select times and latitudes
    ds_selected = ds_merged.sel(time =  ds_merged.time.dt.month.isin(month_list),
                                lat = slice(filter_dict["min_lat"], filter_dict["max_lat"]))
    print_ds_info(ds_selected, ds_var_name, print_counts)
    
    # mask depths
    ds_masked = ds_selected.where(
        (ds_selected.deptho >= filter_dict["min_depth"]) & 
        (ds_selected.deptho <= filter_dict["max_depth"])
    )
    print_ds_info(ds_masked, ds_var_name, print_counts)
    
    # calculate mean
    ds_mean = ds_masked[ds_var_name].groupby('time.year').mean(dim = ['time', 'lon', 'lat'])
    
    # create pandas dataframe
    ds_df = ds_mean.to_dataframe()
    
    # rename column from dataset to dataframe variable name
    ds_df.rename(columns = {ds_var_name:df_var_name}, inplace = True)

    # end timer and print runtime
    end_time = time.time()
    print(f'Runtime: {round((end_time - start_time) / 60, 2)} minutes')

    # write output
    ds_df.to_csv("../../Data/MOM6/yellowtail_MOM6_" + df_var_name + ".csv")
    
    return ds_df

In [6]:
'''
Calculate annual mean for a 3-D variable

Inputs:
- var_dataset: xarray Dataset with the variable to average
- depth_dataset: xarray Dataset with depth to filter
- ds_var_name: string with the variable name, used to extract from var_dataset
- df_var_name: string with the variable name, used to rename column in returned dataframe
- filter_dict: dictionary with relevant filters
    - min_mon, max_mon: minimum and maximum month (integers, inclusive)
    - min_lat, max_lat: minimum and maximum latitude (integers or floats, inclusive)
    - min_depth, max_depth: minimum and maximum depth to filter the longitudinal extent
- print_counts: boolean, to print record count and % at each step; defaults to False for 3-D

Outputs:
- ds_df: pandas dataframe with an annual average of the variable
'''

def annual_mean_3d(var_dataset, depth_dataset, ds_var_name, df_var_name, filter_dict, print_counts = False):

    # start timer
    start_time = time.time()
    
    print("---" + df_var_name + "---")
    print_ds_info(var_dataset, ds_var_name, print_counts)
    
    # merge files
    ds_merged = xr.merge([var_dataset, depth_dataset])
    print_ds_info(ds_merged, ds_var_name, print_counts)

    # create list with months
    month_list = [x for x in range(filter_dict["min_month"], filter_dict["max_month"]+1)]
    # select times and latitudes
    ds_selected = ds_merged.sel(time =  ds_merged.time.dt.month.isin(month_list),
                                lat = slice(filter_dict["min_lat"], filter_dict["max_lat"]))
    print_ds_info(ds_selected, ds_var_name, print_counts)
    
    # mask depths
    ds_masked = ds_selected.where(
        (ds_selected.deptho >= filter_dict["min_depth"]) & 
        (ds_selected.deptho <= filter_dict["max_depth"])
    )
    print_ds_info(ds_masked, ds_var_name, print_counts)
    
    # calculate mean
    ds_mean = ds_masked[ds_var_name].groupby('time.year').mean(dim = ['time', 'z_l', 'lon', 'lat'])
    
    # create pandas dataframe
    ds_df = ds_mean.to_dataframe()
    
    # rename column from dataset to dataframe variable name
    ds_df.rename(columns = {ds_var_name:df_var_name}, inplace = True)

    # end timer and print runtime
    end_time = time.time()
    print(f'Runtime: {round((end_time - start_time) / 60, 2)} minutes')

    # write output
    ds_df.to_csv("../../Data/MOM6/yellowtail_MOM6_" + df_var_name + ".csv")
    
    return ds_df

## Calculate means

In [7]:
# mixed layer depths
MLDpart = annual_mean_2d(ds_mld, ds_depth, "MLD_003", "MLDpart", lifestage_dict['part'], True)
MLDlarv = annual_mean_2d(ds_mld, ds_depth, "MLD_003", "MLDlarv", lifestage_dict['larv'], True)
MLDpjuv = annual_mean_2d(ds_mld, ds_depth, "MLD_003", "MLDpjuv", lifestage_dict['pjuv'], True)

---MLDpart---
106719360 records with dimensions (384, 815, 341)
38069760 records (35.67%) are not np.nan
106719360 records with dimensions (384, 815, 341)
38069760 records (35.67%) are not np.nan
4102912 records with dimensions (128, 94, 341)
1804288 records (43.98%) are not np.nan
4102912 records with dimensions (128, 94, 341)
12416 records (0.30%) are not np.nan
Runtime: 0.02 minutes
---MLDlarv---
106719360 records with dimensions (384, 815, 341)
38069760 records (35.67%) are not np.nan
106719360 records with dimensions (384, 815, 341)
38069760 records (35.67%) are not np.nan
2051456 records with dimensions (64, 94, 341)
902144 records (43.98%) are not np.nan
2051456 records with dimensions (64, 94, 341)
2944 records (0.14%) are not np.nan
Runtime: 0.0 minutes
---MLDpjuv---
106719360 records with dimensions (384, 815, 341)
38069760 records (35.67%) are not np.nan
106719360 records with dimensions (384, 815, 341)
38069760 records (35.67%) are not np.nan
5128640 records with dimensions

In [8]:
# temperatures
Tcop = annual_mean_3d(ds_temp, ds_depth, "thetao", "Tcop", lifestage_dict['cop'], False)
Tpart = annual_mean_3d(ds_temp, ds_depth, "thetao", "Tpart", lifestage_dict['part'], False)

---Tcop---
5549406720 records with dimensions (384, 52, 815, 341)
5549406720 records with dimensions (384, 52, 815, 341)
160013568 records with dimensions (96, 52, 94, 341)
160013568 records with dimensions (96, 52, 94, 341)
Runtime: 12.92 minutes
---Tpart---
5549406720 records with dimensions (384, 52, 815, 341)
5549406720 records with dimensions (384, 52, 815, 341)
213351424 records with dimensions (128, 52, 94, 341)
213351424 records with dimensions (128, 52, 94, 341)
Runtime: 17.08 minutes


In [9]:
# X sea water velocities
XVlarv = annual_mean_3d(ds_xv, ds_depth, "uo_rotate", "XVlarv", lifestage_dict['larv'], False)
XVpjuv = annual_mean_3d(ds_xv, ds_depth, "uo_rotate", "XVpjuv", lifestage_dict['pjuv'], False)

---XVlarv---
5549406720 records with dimensions (384, 52, 815, 341)
5549406720 records with dimensions (384, 52, 815, 341)
106675712 records with dimensions (64, 52, 94, 341)
106675712 records with dimensions (64, 52, 94, 341)
Runtime: 9.02 minutes
---XVpjuv---
5549406720 records with dimensions (384, 52, 815, 341)
5549406720 records with dimensions (384, 52, 815, 341)
266689280 records with dimensions (160, 52, 94, 341)
266689280 records with dimensions (160, 52, 94, 341)
Runtime: 22.6 minutes


In [10]:
# Y sea water velocities
YVlarv = annual_mean_3d(ds_yv, ds_depth, "vo_rotate", "YVlarv", lifestage_dict['larv'], False)
YVpjuv = annual_mean_3d(ds_yv, ds_depth, "vo_rotate", "YVpjuv", lifestage_dict['pjuv'], False)

---YVlarv---
5549406720 records with dimensions (384, 52, 815, 341)
5549406720 records with dimensions (384, 52, 815, 341)
106675712 records with dimensions (64, 52, 94, 341)
106675712 records with dimensions (64, 52, 94, 341)
Runtime: 9.15 minutes
---YVpjuv---
5549406720 records with dimensions (384, 52, 815, 341)
5549406720 records with dimensions (384, 52, 815, 341)
266689280 records with dimensions (160, 52, 94, 341)
266689280 records with dimensions (160, 52, 94, 341)
Runtime: 22.7 minutes


## Lag indices as needed

For life stages where data from the prior calendar year are relevant for this year's recruitment deviaitons, adjust the index +1. For yellowtail, those life stages are preconditioning, copulation, and egg fertilization.

In the future, it'd be nice to do this programatically by adding the lag to the lifestage dictionary.

In [11]:
# lag computation data by one year
Tcop.index += 1

## Combine and write output

In [13]:
# merge pandas dataframes
output_df = pd.concat([MLDpart, MLDlarv, MLDpjuv, Tcop, Tpart, XVlarv, XVpjuv, YVlarv, YVpjuv], axis = 1)

# sort indices
output_df.sort_index(inplace = True)

# save result
output_df.to_csv("../../Data/MOM6/yellowtail_MOM6.csv")

# preview result
output_df

,MLDpart,MLDlarv,MLDpjuv,Tcop,Tpart,XVlarv,XVpjuv,YVlarv,YVpjuv
year,,,,,,,,,
1993,11.830518,10.904479,6.428638,NaN,9.837281,-0.000976,-0.000743,0.028028,-0.039921
1994,11.643274,11.090589,6.449591,9.238317,9.676904,0.001127,-0.000629,0.024087,-0.078326
1995,10.150390,8.718237,6.199258,9.172405,9.853590,-0.000158,-0.004624,0.036343,-0.073177
1996,8.544428,7.829722,6.873740,8.946812,10.091395,0.003210,-0.001964,0.018963,-0.061716
1997,8.017573,7.474869,5.906194,8.820283,9.323155,0.001179,-0.004516,-0.015980,-0.057038
1998,11.557767,8.623075,6.338513,10.290700,11.195883,0.000093,-0.005676,0.038934,-0.058639
1999,10.015160,8.873915,6.231465,9.537517,9.113604,0.001303,-0.004414,0.068168,-0.081536
2000,9.018702,7.800598,6.196394,8.594275,9.422470,-0.000386,-0.003483,0.017836,-0.062540
2001,15.199876,14.303400,6.721168,8.697865,9.137657,-0.000204,-0.003639,-0.001160,-0.067408
